In [1]:
import sys
import numpy as np
import pandas as pd
sys.path.append("../code/")

In [2]:
from dataloader import APPLIANCE_ORDER, get_train_test
from ddsc import SparseCoding, reshape_for_sc

In [3]:
num_folds = 5

In [9]:
train[:, 1:, :, :].mean(axis=0).mean(axis=1).shape

(5, 24)

In [10]:
train, test = get_train_test(num_folds=num_folds, fold_num=0)
pred = np.zeros_like(test[:, 1:, :, :])
pred[:, :, :, :] = train[:, 1:, :, :].mean(axis=0).mean(axis=1)

ValueError: could not broadcast input array from shape (5,24) into shape (14,5,112,24)

In [13]:
appliance, day=1, 2
train[:, appliance+1, day, :].mean(axis=0)

array([  83.50956826,   77.5453702 ,   71.14043236,   80.12469158,
         74.25370374,   73.06944454,   83.01080241,   87.28919759,
         83.57160514,   92.62314825,   88.98024672,   82.98888863,
         86.47160493,   81.89876531,   82.51697584,   86.35709858,
         88.82592587,  104.5145063 ,   99.73209911,   88.99351833,
         84.62623447,   91.52469175,   91.43024636,   80.11265391])

In [14]:
def mean_predictions():
    out = []
    for cur_fold in range(5):
        train, test = get_train_test(num_folds=num_folds, fold_num=cur_fold)
        pred = np.zeros_like(test[:, 1:, :, :])
        for appliance in range(test.shape[1]-1):
            for day in range(test.shape[2]):
                pred[:, appliance, day, :] = train[:, appliance+1, day, :].mean(axis=0)
        out.append(pred)
    return np.concatenate(out)


In [16]:
tensor = np.load('../2015-5appliances.numpy.npy')
from sklearn.metrics import mean_absolute_error

err_mean ={}
gt = tensor[:, 1:, :, :]
pred = mean_predictions()
# Clamping prediction to aggregate
pred = np.minimum(pred, tensor[:, 0:1, :,:])
err_mean = {APPLIANCE_ORDER[i+1]:mean_absolute_error(pred[:, i,:,:].flatten(), 
                                                                       gt[:, i, :, :].flatten()) for i in range(pred.shape[1])}

In [17]:
pd.Series(err_mean)

dr        117.771762
dw         28.627193
fridge     28.091365
hvac      316.009448
mw         11.219023
dtype: float64